In [2]:
import os
import pandas as pd
import time
import datetime
import seaborn
import matplotlib.pyplot as plt 
import bs4

In [12]:
from selenium import webdriver     
browser = webdriver.Chrome('chromedriver.exe')   

url = 'https://www.zomato.com/bangalore/great-food-no-bull'
browser.get(url)

parents = browser.find_elements_by_css_selector('div.relative.top-res-box.entity-ads-snippet-track')

# 1.	From this URL (https://www.zomato.com/bangalore/great-food-no-bull),
# scrape the following information and store them in a data frame
-	Restaurant Name
-	URL to the restaurant

In [14]:
zomato_data = pd.DataFrame()
for p in parents:
    restaurant_name = p.find_element_by_css_selector('div.res_title')
    restaurant_name = restaurant_name.get_attribute('innerHTML').replace('\n','')
    
    restaurant_url = p.find_element_by_css_selector('a.ptop0').get_attribute('href')
    
    restaurant_id = p.get_attribute('data-res-id')
    
    #print(restaurant_name)
    #print(restaurant_url)
    zomato_data = zomato_data.append({'Restaurant':restaurant_name,
                                      'Restaurant-URL':restaurant_url,
                                      'Restaurant ID':restaurant_id},ignore_index=True)
zomato_data.head()

Restaurant Restaurant ID  \
0                The Terrace at Gilly's...                18586207   
1                              Burma Burma                18625420   
2                         Output Bengaluru                18704795   
3              3B's - Buddies, Bar &amp;...      ...      18202991   
4                                    Roots                18270163   

                                      Restaurant-URL  
0  https://www.zomato.com/bangalore/the-terrace-a...  
1  https://www.zomato.com/bangalore/burma-burma-i...  
2  https://www.zomato.com/bangalore/output-bengal...  
3  https://www.zomato.com/bangalore/3bs-buddies-b...  
4  https://www.zomato.com/bangalore/roots-koraman...

# 2.	Loop through each restaurant and do the following
- a.	Check if you are able to scrape sample reviews. Because certain URLs will take you to restaurant’s branches page. You can skip these kind of restaurants 
- b.	If you are able to scrape sample reviews, make sure you click the load more button continuously in a loop until all reviews are loaded
# c.	From each review extract the following
- i.	Restaurant name
- ii.	Restaurant ID
- iii.	Reviewer ID
- iv.	Reviewer name
- v.	Followers for the reviewer
- vi.	Total reviews by the user
- vii.	Ratings
- viii.	Review
- ix.	Review date
- x.	Likes

In [11]:
zomato_data1 = pd.DataFrame()
for i,row in zomato_data.head(5).iterrows():
    url1 = row['Restaurant-URL']
    browser.get(url1)
    
    while True:
        try:
            browser.find_element_by_css_selector('div.load-more').click()
            time.sleep(3)
        except:
            break
        
    parent1 = browser.find_elements_by_css_selector('div.ui.segments.res-review-body.res-review.clearfix')
    #print(len(parent1))
    for p1 in parent1:
        try:
            sample_review = p1.find_element_by_css_selector('div.rev-text')
            sample_review = sample_review.get_attribute('innerHTML').replace('RATED','')
            sample_review = bs4.BeautifulSoup(sample_review).text.replace('\n','')
            
            reviewer_name = p1.find_element_by_css_selector('div.header.nowrap.ui.left')
            reviewer_name = reviewer_name.find_element_by_css_selector('a').get_attribute('innerHTML').replace('\n','')
            
            reviewer_id = p1.get_attribute('data-review_id')
            
            reviewer_rating = p1.find_element_by_css_selector('div.ttupper').get_attribute('aria-label').replace('Rated','')
            
            review_date = p1.find_element_by_css_selector('a.grey-text')
            review_date = review_date.find_element_by_css_selector('time').get_attribute('datetime').replace('\n','')
            
            reviewer_followers = p1.find_element_by_css_selector('span.grey-text.fontsize5.nowrap').get_attribute('innerHTML').split(',')[1].replace('Followers','').replace('\n','')
            reviewer_ratings = p1.find_element_by_css_selector('span.grey-text.fontsize5.nowrap').get_attribute('innerHTML').split(',')[0].replace('Reviews','').replace('\n','')
            
            review_like = p1.find_element_by_css_selector('div.ui.basic.label.stats-thanks.js-thank-count.js-activity-like-count').get_attribute('aria-label').replace('Likes','')
            
            restaurant_name = row['Restaurant']
            
            restaurant_ID = row['Restaurant ID']
            
            zomato_data1 = zomato_data1.append({'Restaurant Name':restaurant_name,
                                                'Restaurant id':restaurant_ID,
                                              'Name Of Reviewer':reviewer_name,
                                              'Reviewer ID':reviewer_id,
                                              'Review':sample_review,
                                              'Reviewer Rating':reviewer_rating,
                                              'Date Of Review':review_date,
                                              'Followers':reviewer_followers,
                                              'Likes':review_like,
                                              'Total Ratings given':reviewer_ratings},ignore_index=True)
        except:
            pass
        
        #print(restaurant_name)
        #print(reviewer_ratings)
        #print(sample_review)
        #print(reviewer_name)
        #print(reviewer_id)

        zomato_data1

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Date Of Review                                          Followers  \
0    2018-12-20 14:14:18                            1316                ...   
1    2018-12-18 07:45:32                            21                  ...   
2    2018-12-17 15:13:50                            369                 ...   
3    2018-12-16 12:08:11                            4428                ...   
4    2018-12-16 01:43:41                            1856                ...   
5    2018-12-15 21:01:20                            1367                ...   
6    2018-12-15 17:08:35                            459                 ...   
7    2018-12-14 01:06:47                            1123                ...   
8    2018-12-13 07:53:03                            414                 ...   
9    2018-12-13 01:17:33                            188                 ...   
10   2018-09-11 16:42:07                            401                 ...   
11   2018-09-07 22:38:38                            156                 ...   
12   2018-09-01 15:51:29                            25                  ...   
13   2018-08-19 21:28:18                            170                 ...   
14   2018-08-13 12:12:11                            203                 ...   
15   2018-08-12 15:55:15                            17                  ...   
16   2018-08-01 00:02:17                            217                 ...   
17   2018-07-23 01:19:36                            13                  ...   
18   2018-07-20 18:20:11                            707                 ...   
19   2018-07-17 09:54:37                            12403               ...   
20   2018-07-07 22:00:49                            149                 ...   
21   2018-06-29 20:07:25                            588                 ...   
22   2018-06-29 15:43:05                            163                 ...   
23   2018-06-26 19:34:35                            108                 ...   
24   2018-06-25 09:27:00                            109                 ...   
25   2018-12-20 18:53:30                            147                 ...   
26   2018-12-16 23:11:37                            35                  ...   
27   2018-12-16 18:56:16                            526                 ...   
28   2018-12-15 20:53:11                            368                 ...   
29   2018-12-15 14:47:08                            53                  ...   
..                   ...                                                ...   
206  2018-07-21 15:17:26                            51                  ...   
207  2018-07-18 14:13:10                            47                  ...   
208  2018-07-18 02:11:42                            305                 ...   
209  2018-07-17 22:50:01                            155                 ...   
210  2018-07-17 20:14:30                            226                 ...   
211  2018-07-17 18:12:46                            92                  ...   
212  2018-07-17 10:56:30                            1903                ...   
213  2018-07-16 21:42:24                            1610                ...   
214  2018-07-16 11:29:53                            231                 ...   
215  2018-07-11 17:51:24                            323                 ...   
216  2018-07-10 17:19:01                            101                 ...   
217  2018-07-06 00:08:50                            232                 ...   
218  2018-07-05 12:33:30                            165                 ...   
219  2018-07-05 08:40:01                            951                 ...   
220  2018-07-03 19:24:14                            69                  ...   
221  2018-07-02 21:59:24                            130                 ...   
222  2018-07-02 16:55:11                            319                 ...   
223  2018-07-01 13:17:53                            738                 ...   
224  2018-06-24 20:46:05                            1042    